# Performance dataframes

In [1]:
import os
import re
import json
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
from natsort import index_natsorted

/Users/andersvandvik/Repositories/masters-thesis/venv/lib/python3.9/site-packages/pandas/compat/__init__.py:97: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


## General parameters

In [2]:
nbr_sims = 5

instance_key = 'instance'
best_obj_key = 'best obj'
worst_obj_key = 'worst obj'
avg_obj_key = 'obj'
max_time_key = 'max time'
min_time_key = 'min time'
avg_time_key = 'time'
max_iter_key = 'max iter'
min_iter_key = 'min iter'
avg_iter_key = 'iter'
set_part_key = 'sp improv'

gap_key = 'gap'
preprocess_key = 'preproc'
model_key = 'model time'
variables_key = 'variables'


project_path = os.path.dirname(os.path.abspath('.'))
directory_path_alns = '/output/solstorm/alns/performance/'
directory_path_exact = '/output/solstorm/arcflow/performance/'

generate_df = True
run_number = 'fourth'

## Functions

In [3]:
def map_instance_to_data_alns(run_path):
    instance_to_data = {}
    for file_name in os.listdir(run_path):
        split_name = re.split('_|\.', file_name)
        instance_name = split_name[0]
        is_history = split_name[2] == 'history'
        if is_history:
            with open(run_path + file_name) as file:
                history_json = json.load(file)
        
            obj = history_json['best_objective']
            time = history_json['runtime']
            it = history_json['number_of_iterations']
            set_part_improv = history_json['number_of_improvements_by_set_partitioning']

            if instance_name in instance_to_data:
                data = instance_to_data[instance_name]

                if obj < data[0]:
                    data[0] = obj
                if obj > data[1]:
                    data[1] = obj
                if time > data[3]:
                    data[3] = time
                if time < data[4]:
                    data[4] = time
                if it > data[6]:
                    data[6] = it
                if it < data[7]:
                    data[7] = it

                data[2] += obj
                data[5] += time
                data[8] += it
                data[9] += set_part_improv
                data[10] += 1
            else:
                instance_to_data[instance_name] = [obj, obj, obj, time, time, time, it, it, it, set_part_improv, 1]
    return instance_to_data

def generate_run_df_alns(run_name):
    run_path = project_path + directory_path_alns + run_name
    instance_to_data = map_instance_to_data_alns(run_path)
    
    df = pd.DataFrame(columns=[instance_key, 
                               best_obj_key, worst_obj_key, avg_obj_key, 
                               max_time_key, min_time_key, avg_time_key, 
                               max_iter_key, min_iter_key, avg_iter_key])
    
    for instance in instance_to_data:
        data = instance_to_data[instance]
        best_objective = data[0]
        worst_objective = data[1]
        avg_objective = data[2] / nbr_sims
        max_time = data[3]
        min_time = data[4]
        avg_time = data[5] / nbr_sims
        max_iter = data[6]
        min_iter = data[7]
        avg_iter = data[8] / nbr_sims
        avg_set_part_improv = data[9] / nbr_sims
        
        if data[10] != nbr_sims:
            print(f'{instance} HAS LESS THAN FIVE SIMULATIONS!')
        
        row = pd.Series({instance_key: instance, 
                         best_obj_key: best_objective,
                         worst_obj_key: worst_objective,
                         avg_obj_key: avg_objective, 
                         max_time_key: max_time,
                         min_time_key: min_time,
                         avg_time_key: avg_time, 
                         max_iter_key: max_iter,
                         min_iter_key: min_iter,
                         avg_iter_key: avg_iter,
                         set_part_key: avg_set_part_improv})
        
        df = df.append(row, ignore_index=True)
    
    df = df.sort_values(by='instance',
                        key=lambda x: np.argsort(index_natsorted(df['instance'])),
                        inplace=False)
    df = df.reset_index(drop=True)
    
    mean_row = pd.Series({instance_key: 'Mean values',
                          best_obj_key: df[best_obj_key].mean(),
                          worst_obj_key: df[worst_obj_key].mean(),
                          avg_obj_key: df[avg_obj_key].mean(),
                          max_time_key: df[max_time_key].mean(),
                          min_time_key: df[min_time_key].mean(),
                          avg_time_key: df[avg_time_key].mean(),
                          max_iter_key: df[max_iter_key].mean(),
                          min_iter_key: df[min_iter_key].mean(),
                          avg_iter_key: df[avg_iter_key].mean(),
                          set_part_key: df[set_part_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def map_instance_to_data_exact(run_path):
    instance_to_data = {}
    for file_name in os.listdir(run_path):
        split_name = re.split('_|\.', file_name)
        instance_name = split_name[0]
        with open(run_path + file_name) as file:
            exact_json = json.load(file)

        obj = exact_json['objective']['objective_bound']
        gap = exact_json['objective']['optimality_gap']
        model_runtime = exact_json['runtime']['model_runtime']
        preprocess_runtime = exact_json['runtime']['preprocess_runtime']
        variables = exact_json['variables']['number_of_variables']
        
        if instance_name in instance_to_data:
            print('Multiple versions of same instance!')
        
        instance_to_data[instance_name] = [obj, gap, model_runtime, preprocess_runtime, variables]
    
    return instance_to_data

def generate_run_df_exact(run_name):
    run_path = project_path + directory_path_exact + run_name
    instance_to_data = map_instance_to_data_exact(run_path)

    df = pd.DataFrame(columns=[instance_key, avg_obj_key, gap_key, preprocess_key, model_key, variables_key])
    for instance in instance_to_data:
        data = instance_to_data[instance]
        obj = data[0]
        gap = data[1]
        preprocess_runtime = data[2]
        model_runtime = data[3]
        variables = data[4]
        row = pd.Series({instance_key: instance,
                         avg_obj_key: obj,
                         gap_key: gap,
                         preprocess_key: preprocess_runtime,
                         model_key: model_runtime,
                         variables_key: variables})
        df = df.append(row, ignore_index=True)
    
    df = df.sort_values(by='instance',
                        key=lambda x: np.argsort(index_natsorted(df['instance'])),
                        inplace=False)
    df = df.reset_index(drop=True)
    
    mean_row = pd.Series({instance_key: 'Mean values',
                          avg_obj_key: df[avg_obj_key].mean(),
                          gap_key: df[gap_key].mean(),
                          preprocess_key: df[preprocess_key].mean(),
                          model_key: df[model_key].mean(),
                          variables_key: df[variables_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def merge_dfs(dfs, drop):
    df_copies = [df.copy() for df in dfs]
    for df in df_copies:
        df.drop([best_obj_key, worst_obj_key, max_time_key, min_time_key, max_iter_key, min_iter_key], 
                axis=1, inplace=True)
    df_total = pd.concat(df_copies, axis=1)
    
    # Drop duplicate instance columns
    if drop:
        li = [i for i in range(5, len(df_total.columns), 5)]
        df_total = df_total.iloc[:, [j for j, c in enumerate(df_total.columns) if j not in li]]
    
    df_total = df_total.round(1)
    return df_total

def aggregate_df_by_instance_group_alns(df):
    instance_size_to_data = {}
    for idx, row in df.iterrows():
        instance_name = row[instance_key]

        if instance_name == 'Mean values':
            continue
        
        split_name = re.split('-', instance_name)
        instance_size = split_name[0]
        
        best_obj = row[best_obj_key]
        worst_obj = row[worst_obj_key]
        avg_obj = row[avg_obj_key]
        max_time = row[max_time_key]
        min_time = row[min_time_key]
        avg_time = row[avg_time_key]
        max_iter = row[max_iter_key]
        min_iter = row[min_iter_key]
        avg_iter = row[avg_iter_key]
        avg_set_part_improv = row[set_part_key]
        
        if instance_size in instance_size_to_data:
            data = instance_size_to_data[instance_size]
            data[0] += best_obj
            data[1] += worst_obj
            data[2] += avg_obj
            data[3] += max_time
            data[4] += min_time
            data[5] += avg_time
            data[6] += max_iter
            data[7] += min_iter
            data[8] += avg_iter
            data[9] += avg_set_part_improv
            data[10] += 1  # Number of times encountered instance size
        else:
            data = [best_obj, worst_obj, avg_obj, max_time, min_time, avg_time, max_iter, min_iter, avg_iter, avg_set_part_improv, 1]
            instance_size_to_data[instance_size] = data

    df = pd.DataFrame(columns=['instance_group', best_obj_key, worst_obj_key, avg_obj_key, max_time_key, min_time_key,
                               avg_time_key, max_iter_key, min_iter_key, avg_iter_key, set_part_key])
    
    for instance_size in instance_size_to_data:
        data = instance_size_to_data[instance_size]
        nbr_sims = data[10]
        if nbr_sims < 5:
            print(f'{instance_size} INSTANCE_SIZE LESS THAN FIVE INSTANCES!')
        
        row = pd.Series({'instance_group': instance_size, 
                         best_obj_key: data[0] / nbr_sims,
                         worst_obj_key: data[1] / nbr_sims,
                         avg_obj_key: data[2] / nbr_sims, 
                         max_time_key: data[3] / nbr_sims,
                         min_time_key: data[4] / nbr_sims,
                         avg_time_key: data[5] / nbr_sims, 
                         max_iter_key: data[6] / nbr_sims,
                         min_iter_key: data[7] / nbr_sims,
                         avg_iter_key: data[8] / nbr_sims,
                         set_part_key: data[9] / nbr_sims})
    
        df = df.append(row, ignore_index=True)
        
    mean_row = pd.Series({'instance_group': 'Mean values',
                          best_obj_key: df[best_obj_key].mean(),
                          worst_obj_key: df[worst_obj_key].mean(),
                          avg_obj_key: df[avg_obj_key].mean(),
                          max_time_key: df[max_time_key].mean(),
                          min_time_key: df[min_time_key].mean(),
                          avg_time_key: df[avg_time_key].mean(),
                          max_iter_key: df[max_iter_key].mean(),
                          min_iter_key: df[min_iter_key].mean(),
                          avg_iter_key: df[avg_iter_key].mean(),
                          set_part_key: df[set_part_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def aggregate_df_by_instance_group_exact(df):
    instance_size_to_data = {}
    for idx, row in df.iterrows():
        instance_name = row[instance_key]

        if instance_name == 'Mean values':
            continue
        
        split_name = re.split('-', instance_name)
        instance_size = split_name[0]
        
        obj = row[avg_obj_key]
        gap = row[gap_key]
        preprocess_runtime = row[preprocess_key]
        model_runtime = row[model_key]
        variables = row[variables_key]
        
        if instance_size in instance_size_to_data:
            data = instance_size_to_data[instance_size]
            data[0] += obj
            data[1] += gap
            data[2] += preprocess_runtime
            data[3] += model_runtime
            data[4] += variables
            data[5] += 1 # Number of times encountered instance size
        else:
            instance_size_to_data[instance_size] = [obj, gap, preprocess_runtime, model_runtime, variables, 1]

    df = pd.DataFrame(columns=['instance_group', avg_obj_key, gap_key, preprocess_key, model_key, variables_key])
    
    for instance_size in instance_size_to_data:
        data = instance_size_to_data[instance_size]
        nbr_sims = data[5]
        if nbr_sims < 5:
            print(f'{instance_size} INSTANCE_SIZE LESS THAN FIVE INSTANCES!')
        
        row = pd.Series({'instance_group': instance_size, 
                         avg_obj_key: data[0] / nbr_sims,
                         gap_key: data[1] / nbr_sims,
                         preprocess_key: data[2] / nbr_sims, 
                         model_key: data[3] / nbr_sims,
                         variables_key: data[4] / nbr_sims})
    
        df = df.append(row, ignore_index=True)

        
    mean_row = pd.Series({'instance_group': 'Mean values', 
                          avg_obj_key: df[avg_obj_key].mean(),
                          gap_key: df[gap_key].mean(),
                          preprocess_key: df[preprocess_key].mean(), 
                          model_key: df[model_key].mean(),
                          variables_key: df[variables_key].mean()})
    df = df.append(mean_row, ignore_index=True)
    df = df.round(1)
    return df

def load_df(file_name):
    run_df = pd.read_pickle(f'dataframes/performance/{file_name}')
    # run_df = sort_df(run_df, sort_column)
    return run_df

## ALNS baseline

In [4]:
if generate_df:
    run_baseline_name = f'{run_number}/baseline/'
    run_baseline_df = generate_run_df_alns(run_baseline_name)
    run_baseline_agg_df = aggregate_df_by_instance_group_alns(run_baseline_df)

    run_baseline_file_name = f'dataframes/performance/baseline.pkl'
    run_baseline_agg_file_name = f'dataframes/performance/baseline_agg.pkl'
    run_baseline_df.to_pickle(run_baseline_file_name)
    run_baseline_agg_df.to_pickle(run_baseline_agg_file_name)

25-30-4-1 HAS LESS THAN FIVE SIMULATIONS!
21-23-3-2 HAS LESS THAN FIVE SIMULATIONS!
25 INSTANCE_SIZE LESS THAN FIVE INSTANCES!


In [5]:
run_baseline_df = load_df('baseline.pkl')
run_baseline_df

,instance,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,11-12-2-1,3587.5,3587.5,3587.5,19.6,17.9,18.6,5000,5000,5000.0,0.0
1,11-12-2-2,3788.9,3788.9,3788.9,15.8,14.0,15.0,5000,5000,5000.0,0.0
2,11-13-2-1,3470.8,3470.8,3470.8,20.1,18.9,19.4,5000,5000,5000.0,0.0
3,11-14-2-1,3710.8,3710.8,3710.8,25.6,21.2,23.2,5000,5000,5000.0,0.0
4,11-15-2-1,3029.7,3029.7,3029.7,18.4,16.8,17.8,5000,5000,5000.0,0.0
5,13-14-2-1,3681.9,3681.9,3681.9,35.0,29.7,32.3,5000,5000,5000.0,0.0
6,13-15-2-1,4122.6,4122.6,4122.6,35.1,31.6,33.6,5000,5000,5000.0,0.0
7,13-15-2-2,3779.4,3779.4,3779.4,22.2,19.2,20.7,5000,5000,5000.0,0.0
8,13-16-2-1,4239.6,4239.6,4239.6,36.2,31.7,33.7,5000,5000,5000.0,0.0
9,13-18-2-1,4029.1,4029.1,4029.1,47.3,39.6,42.2,5000,5000,5000.0,0.0


In [6]:
run_baseline_agg_df = load_df('baseline_agg.pkl')
run_baseline_agg_df

,instance_group,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,11,3517.5,3517.5,3517.5,19.9,17.8,18.8,5000.0,5000.0,5000.0,0.0
1,13,3970.5,3970.5,3970.5,35.2,30.4,32.5,5000.0,5000.0,5000.0,0.0
2,15,8375.4,8429.6,8389.0,40.3,34.2,37.3,5000.0,5000.0,5000.0,0.0
3,17,4966.1,5000.3,4976.7,61.8,54.3,57.9,5000.0,5000.0,5000.0,0.0
4,19,5153.5,5185.3,5167.4,77.6,67.3,72.6,5000.0,5000.0,5000.0,0.0
5,21,9299.5,9329.7,9118.2,90.0,78.0,80.3,5000.0,5000.0,4800.0,0.0
6,23,5962.1,6024.2,5983.2,139.6,122.4,130.5,5000.0,5000.0,5000.0,0.0
7,25,6517.1,6537.9,5361.2,145.6,135.5,101.4,5000.0,5000.0,4000.0,0.0
8,Mean values,5970.2,5999.4,5810.5,76.2,67.5,66.4,5000.0,5000.0,4850.0,0.0


## Sequential ALNS

In [ ]:
if generate_df:
    run_sequential_name = 'first/sequential/'
    run_sequential_df = generate_run_df_alns(run_sequential_name)
    run_sequential_agg_df = aggregate_df_by_instance_group_alns(run_sequential_df)

    run_sequential_file_name = f'dataframes/performance/sequential.pkl'
    run_sequential_agg_file_name = f'dataframes/performance/sequential_agg.pkl'
    run_sequential_df.to_pickle(run_sequential_file_name)
    run_sequential_agg_df.to_pickle(run_sequential_agg_file_name)

In [ ]:
run_sequential_df = load_df('sequential.pkl')
run_sequential_df

In [ ]:
run_sequential_agg_df = load_df('sequential_agg.pkl')
run_sequential_agg_df

## LNS

In [7]:
if generate_df:
    run_lns_name = f'{run_number}/lns/'
    run_lns_df = generate_run_df_alns(run_lns_name)
    run_lns_agg_df = aggregate_df_by_instance_group_alns(run_lns_df)

    run_lns_file_name = f'dataframes/performance/lns.pkl'
    run_lns_agg_file_name = f'dataframes/performance/lns_agg.pkl'
    run_lns_df.to_pickle(run_lns_file_name)
    run_lns_agg_df.to_pickle(run_lns_agg_file_name)

25-30-4-1 HAS LESS THAN FIVE SIMULATIONS!
11-12-2-2 HAS LESS THAN FIVE SIMULATIONS!
25 INSTANCE_SIZE LESS THAN FIVE INSTANCES!


In [8]:
run_lns_df = load_df('lns.pkl')
run_lns_df

,instance,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,11-12-2-1,3587.5,3587.5,3587.5,19.6,18.2,19.0,5000,5000,5000.0,0.0
1,11-12-2-2,3788.9,3788.9,3031.2,17.4,14.7,12.7,5000,5000,4000.0,0.0
2,11-13-2-1,3470.8,3470.8,3470.8,21.0,18.6,19.7,5000,5000,5000.0,0.0
3,11-14-2-1,3710.8,3710.8,3710.8,22.6,19.7,21.7,5000,5000,5000.0,0.0
4,11-15-2-1,3029.7,3029.7,3029.7,18.2,15.3,17.0,5000,5000,5000.0,0.0
5,13-14-2-1,3681.9,3681.9,3681.9,32.4,30.8,31.4,5000,5000,5000.0,0.0
6,13-15-2-1,4122.6,4122.6,4122.6,33.4,31.8,32.3,5000,5000,5000.0,0.0
7,13-15-2-2,3779.4,3779.4,3779.4,22.9,20.3,21.9,5000,5000,5000.0,0.0
8,13-16-2-1,4239.6,4239.6,4239.6,37.5,31.0,34.4,5000,5000,5000.0,0.0
9,13-18-2-1,4029.1,4029.1,4029.1,47.9,38.1,41.6,5000,5000,5000.0,0.0


In [9]:
run_lns_agg_df = load_df('lns_agg.pkl')
run_lns_agg_df

,instance_group,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,11,3517.5,3517.5,3366.0,19.8,17.3,18.0,5000.0,5000.0,4800.0,0.0
1,13,3970.5,3970.5,3970.5,34.8,30.4,32.3,5000.0,5000.0,5000.0,0.0
2,15,8375.4,8427.9,8400.4,40.1,34.1,37.6,5000.0,5000.0,5000.0,0.0
3,17,4966.1,4990.1,4974.4,60.5,51.8,55.8,5000.0,5000.0,5000.0,0.0
4,19,5153.5,5185.8,5166.5,77.6,66.5,71.4,5000.0,5000.0,5000.0,0.0
5,21,9298.6,9360.3,9327.6,87.5,76.0,82.9,5000.0,5000.0,5000.0,0.0
6,23,5959.4,6032.0,5983.1,134.5,118.0,126.7,5000.0,5000.0,5000.0,0.0
7,25,6504.0,6538.8,5651.2,143.8,132.9,109.0,5000.0,5000.0,4250.0,0.0
8,Mean values,5968.1,6002.9,5855.0,74.8,65.9,66.7,5000.0,5000.0,4881.2,0.0


## ALNS + local search

In [10]:
if generate_df:
    run_ls_name = f'{run_number}/ls/'
    run_ls_df = generate_run_df_alns(run_ls_name)
    run_ls_agg_df = aggregate_df_by_instance_group_alns(run_ls_df)

    run_ls_file_name = f'dataframes/performance/ls.pkl'
    run_ls_agg_file_name = f'dataframes/performance/ls_agg.pkl'
    run_ls_df.to_pickle(run_ls_file_name)
    run_ls_agg_df.to_pickle(run_ls_agg_file_name)

19-21-3-1 HAS LESS THAN FIVE SIMULATIONS!
19 INSTANCE_SIZE LESS THAN FIVE INSTANCES!


In [11]:
run_ls_df = load_df('ls.pkl')
run_ls_df

,instance,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,11-12-2-1,3587.5,3587.5,3587.5,52.4,47.6,50.8,5000,5000,5000.0,0.0
1,11-12-2-2,3788.9,3788.9,3788.9,54.2,49.3,51.9,5000,5000,5000.0,0.0
2,11-13-2-1,3470.8,3470.8,3470.8,65.2,56.4,62.0,5000,5000,5000.0,0.0
3,11-14-2-1,3710.8,3710.8,3710.8,65.8,60.5,62.1,5000,5000,5000.0,0.0
4,11-15-2-1,3029.7,3029.7,3029.7,39.2,32.5,35.6,5000,5000,5000.0,0.0
5,13-14-2-1,3681.9,3681.9,3681.9,99.5,85.9,90.8,5000,5000,5000.0,0.0
6,13-15-2-1,4122.6,4122.6,4122.6,102.1,94.0,97.1,5000,5000,5000.0,0.0
7,13-15-2-2,3779.4,3779.4,3779.4,56.8,38.0,52.3,5000,5000,5000.0,0.0
8,13-16-2-1,4239.6,4239.6,4239.6,94.4,86.6,90.6,5000,5000,5000.0,0.0
9,13-18-2-1,4029.1,4029.1,4029.1,130.5,122.5,126.7,5000,5000,5000.0,0.0


In [12]:
run_ls_agg_df = load_df('ls_agg.pkl')
run_ls_agg_df

,instance_group,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,11,3517.5,3517.5,3517.5,55.4,49.3,52.5,5000.0,5000.0,5000.0,0.0
1,13,3970.5,3970.5,3970.5,96.7,85.4,91.5,5000.0,5000.0,5000.0,0.0
2,15,8375.4,8395.9,8383.1,98.6,82.2,92.5,5000.0,5000.0,5000.0,0.0
3,17,4966.1,4971.6,4967.6,197.8,169.9,186.6,5000.0,5000.0,5000.0,0.0
4,19,5334.1,5352.2,4244.0,313.6,279.7,239.8,5000.0,5000.0,4000.0,0.0
5,Mean values,5232.7,5241.5,5016.6,152.4,133.3,132.6,5000.0,5000.0,4800.0,0.0


## ALNS + set partitioning

In [13]:
if generate_df: 
    run_sp_name = f'{run_number}/sp/'
    run_sp_df = generate_run_df_alns(run_sp_name)
    run_sp_agg_df = aggregate_df_by_instance_group_alns(run_sp_df)

    run_sp_file_name = f'dataframes/performance/sp.pkl'
    run_sp_agg_file_name = f'dataframes/performance/sp_agg.pkl'
    run_sp_df.to_pickle(run_sp_file_name)
    run_sp_agg_df.to_pickle(run_sp_agg_file_name)

23-24-4-1 HAS LESS THAN FIVE SIMULATIONS!
21-26-3-1 HAS LESS THAN FIVE SIMULATIONS!
23 INSTANCE_SIZE LESS THAN FIVE INSTANCES!


In [14]:
run_sp_df = load_df('sp.pkl')
run_sp_df

,instance,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,11-12-2-1,3587.5,3587.5,3587.5,22.7,21.5,22.1,5000,5000,5000.0,0.0
1,11-12-2-2,3788.9,3788.9,3788.9,20.2,17.7,18.7,5000,5000,5000.0,0.2
2,11-13-2-1,3470.8,3470.8,3470.8,26.1,22.2,24.4,5000,5000,5000.0,0.0
3,11-14-2-1,3710.8,3710.8,3710.8,31.2,27.2,29.1,5000,5000,5000.0,0.0
4,11-15-2-1,3029.7,3029.7,3029.7,24.1,18.7,21.2,5000,5000,5000.0,0.0
5,13-14-2-1,3681.9,3681.9,3681.9,40.3,37.2,38.9,5000,5000,5000.0,0.2
6,13-15-2-1,4122.6,4122.6,4122.6,47.6,36.6,43.2,5000,5000,5000.0,0.0
7,13-15-2-2,3779.4,3779.4,3779.4,31.6,26.7,29.6,5000,5000,5000.0,0.0
8,13-16-2-1,4239.6,4239.6,4239.6,47.8,44.5,46.6,5000,5000,5000.0,0.0
9,13-18-2-1,4029.1,4029.1,4029.1,68.9,60.5,64.2,5000,5000,5000.0,0.0


In [15]:
run_sp_agg_df = load_df('sp_agg.pkl')
run_sp_agg_df

,instance_group,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,11,3517.5,3517.5,3517.5,24.9,21.5,23.1,5000.0,5000.0,5000.0,0.0
1,13,3970.5,3970.5,3970.5,47.2,41.1,44.5,5000.0,5000.0,5000.0,0.0
2,15,8375.4,8394.5,8381.2,59.5,47.0,53.7,5000.0,5000.0,5000.0,0.8
3,17,4966.1,4975.0,4969.5,100.0,77.7,89.0,5000.0,5000.0,5000.0,1.3
4,19,5153.5,5163.0,5157.0,150.1,124.4,137.0,5000.0,5000.0,5000.0,1.9
5,21,9260.5,9316.5,8718.1,180.3,146.1,157.4,5000.0,5000.0,4800.0,2.6
6,23,5141.1,5141.1,2056.4,208.8,204.0,82.6,5000.0,5000.0,2000.0,1.0
7,Mean values,5769.2,5782.6,5252.9,110.1,94.5,83.9,5000.0,5000.0,4542.9,1.1


## ALNS + local search + set partitioning

In [16]:
if generate_df:
    run_lssp_name = f'{run_number}/lssp/'
    run_lssp_df = generate_run_df_alns(run_lssp_name)
    run_lssp_agg_df = aggregate_df_by_instance_group_alns(run_lssp_df)

    run_lssp_file_name = f'dataframes/performance/lssp.pkl'
    run_lssp_agg_file_name = f'dataframes/performance/lssp_agg.pkl'
    run_lssp_df.to_pickle(run_lssp_file_name)
    run_lssp_agg_df.to_pickle(run_lssp_agg_file_name)

17-19-3-1 HAS LESS THAN FIVE SIMULATIONS!
17 INSTANCE_SIZE LESS THAN FIVE INSTANCES!


In [17]:
run_lssp_df = load_df('lssp.pkl')
run_lssp_df

,instance,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,11-12-2-1,3587.5,3587.5,3587.5,57.4,50.9,54.8,5000,5000,5000.0,0.0
1,11-12-2-2,3788.9,3788.9,3788.9,65.0,59.0,62.6,5000,5000,5000.0,0.0
2,11-13-2-1,3470.8,3470.8,3470.8,82.6,71.7,78.3,5000,5000,5000.0,0.0
3,11-14-2-1,3710.8,3710.8,3710.8,82.0,76.1,78.9,5000,5000,5000.0,0.0
4,11-15-2-1,3029.7,3029.7,3029.7,51.3,44.5,47.8,5000,5000,5000.0,0.0
5,13-14-2-1,3681.9,3681.9,3681.9,133.6,117.6,123.2,5000,5000,5000.0,0.0
6,13-15-2-1,4122.6,4122.6,4122.6,126.1,114.1,121.0,5000,5000,5000.0,0.2
7,13-15-2-2,3779.4,3779.4,3779.4,80.7,66.9,71.9,5000,5000,5000.0,0.2
8,13-16-2-1,4239.6,4239.6,4239.6,146.5,123.3,134.4,5000,5000,5000.0,0.0
9,13-18-2-1,4029.1,4029.1,4029.1,302.7,226.7,252.2,5000,5000,5000.0,0.2


In [18]:
run_lssp_agg_df = load_df('lssp_agg.pkl')
run_lssp_agg_df

,instance_group,best obj,worst obj,obj,max time,min time,time,max iter,min iter,iter,sp improv
0,11,3517.5,3517.5,3517.5,67.7,60.4,64.5,5000.0,5000.0,5000.0,0.0
1,13,3970.5,3970.5,3970.5,157.9,129.7,140.5,5000.0,5000.0,5000.0,0.1
2,15,8375.4,8375.4,8375.4,161.5,123.8,146.8,5000.0,5000.0,5000.0,0.8
3,17,4667.8,4667.8,4160.2,544.1,494.6,469.8,4246.5,4077.5,3828.6,0.4
4,Mean values,5132.8,5132.8,5005.9,232.8,202.1,205.4,4811.6,4769.4,4707.2,0.3


## Exact solver

In [12]:
if generate_df:
    run_exact_name = 'results/'
    run_exact_df = generate_run_df_exact(run_exact_name)
    run_exact_agg_df = aggregate_df_by_instance_group_exact(run_exact_df)

    run_exact_file_name = f'dataframes/performance/exact.pkl'
    run_exact_agg_file_name = f'dataframes/performance/exact_agg.pkl'
    run_exact_df.to_pickle(run_exact_file_name)
    run_exact_agg_df.to_pickle(run_exact_agg_file_name)

In [13]:
run_exact_df = load_df('exact.pkl')
run_exact_df

,instance,obj,gap,preproc,model time,variables
0,5-5-1-1,2519.9,0.0,1.3,1.5,24808
1,5-5-1-2,2214.2,0.0,3.7,1.5,36406
2,5-6-1-1,1939.6,0.0,4.2,1.8,40610
3,5-7-1-1,2709.7,0.0,2.7,2.3,35034
4,5-7-1-2,1704.6,0.0,16.0,2.7,64346
5,7-8-1-1,2046.9,0.0,15.2,3.5,71164
6,7-8-1-2,1971.3,0.0,65.4,3.7,84072
7,7-8-1-3,2071.9,0.0,9.1,3.0,47880
8,7-9-1-1,2170.4,0.0,26.6,3.5,80958
9,7-9-1-2,2206.1,0.0,38.7,3.7,85362


In [14]:
run_exact_agg_df = load_df('exact_agg.pkl')
run_exact_agg_df

,instance_group,obj,gap,preproc,model time,variables
0,5,2217.6,0.0,5.6,2.0,40240.8
1,7,2093.3,0.0,31.0,3.5,73887.2
2,9,5627.5,0.0,43.7,5.7,124820.4
3,11,3450.2,0.0,2015.2,13.7,309915.6
4,13,3717.5,0.1,3493.6,20.3,447488.4
5,15,7257.5,0.1,3231.8,27.4,606759.0
6,Mean values,4060.6,0.0,1470.1,12.1,267185.2


## Parallel vs. sequential heuristics

In [ ]:
baseline_sequential_df = merge_dfs([run_baseline_df, run_sequential_df], True)
baseline_sequential_agg_df = merge_dfs([run_baseline_agg_df, run_sequential_agg_df], True)

In [ ]:
baseline_sequential_df

In [ ]:
baseline_sequential_agg_df

## ALNS vs. LNS

In [ ]:
baseline_lns_df = merge_dfs([run_baseline_df, run_lns_df], True)
baseline_lns_agg_df = merge_dfs([run_baseline_agg_df, run_lns_agg_df], True)

In [ ]:
baseline_lns_df

In [ ]:
baseline_lns_agg_df

## ALNS vs. ALNS + local search

In [ ]:
baseline_ls_df = merge_dfs([run_baseline_df, run_ls_df], True)
baseline_ls_agg_df = merge_dfs([run_baseline_agg_df, run_ls_agg_df], True)

In [ ]:
baseline_ls_df

In [ ]:
baseline_ls_agg_df

## ALNS vs. ALNS + set partitioning

In [ ]:
baseline_sp_df = merge_dfs([run_baseline_df, run_sp_df], True)
baseline_sp_agg_df = merge_dfs([run_baseline_agg_df, run_sp_agg_df], True)

In [ ]:
baseline_sp_df

In [ ]:
baseline_sp_agg_df

## ALNS vs. ALNS + local search + set partitioning

In [ ]:
baseline_lssp_df = merge_dfs([run_baseline_df, run_lssp_df], True)
baseline_lssp_agg_df = merge_dfs([run_baseline_agg_df, run_lssp_agg_df], True)

In [ ]:
baseline_lssp_df

In [ ]:
baseline_lssp_agg_df